<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />

# Apilando redes recurrentes<a id="top"></a>

<i>Última actualización: 2025-03-03</small></i></div>
***

## Introducción

Las RNN apiladas (del inglés _stacked_) son un tipo de red neuronal formada por múltiples capas de RNN apiladas unas sobre otras. Se ha demostrado que las RNN apiladas mejoran el rendimiento de las RNN en diversas tareas al permitir que la red aprenda representaciones más complejas de datos secuenciales.

En una RNN apilada, la salida de una capa de la RNN se pasa como entrada a la capa siguiente, creando una representación jerárquica de la secuencia.

## Objetivos

En este _notebook_ resolveremos un problema anterior con este tipo de arquitectura, viendo cómo implementar un modelo con varias capas recurrentes superpuestas.

## Bibliotecas y configuración

A continuación importaremos las librerías que se utilizarán a lo largo del cuaderno.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchmetrics
import torchsummary
import torchvision

import utils

También configuraremos algunos parámetros para adaptar la presentación gráfica.

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

Por último, establecemos las constantes de los recursos comunes.

In [ ]:
DATASETS_DIR = './tmp'
BATCH_SIZE = 8192
TRAIN_EPOCHS = 64

***

## Descarga y preproceso de los datos

Una vez más, utilizaremos el conjunto de datos _fashion_ MNIST porque es mola bastante.

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,))
])

train_set = torchvision.datasets.FashionMNIST(
    root=DATASETS_DIR,
    train=True,
    download=True,
    transform=transform,
)
test_set = torchvision.datasets.FashionMNIST(
    root=DATASETS_DIR,
    train=False,
    download=True,
    transform=transform,
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_set, batch_size=BATCH_SIZE, shuffle=True,
)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=BATCH_SIZE, shuffle=False,
)

## Modelo basado en múltiples capas de `RNN`

En una arquitectura multicapa, **cada capa recurrente procesa una secuencia de entradas** y, al configurarlas para que devuelvan la secuencia completa, la siguiente capa también puede recibir una secuencia.

En PyTorch, el comportamiento por defecto de las `RNN` (por ejemplo, `torch.nn.RNN`) es devolver la secuencia completa, es decir, un tensor de forma `(batch_size, sequence_len, num_features)`. Por lo tanto, al apilar capas recurrentes, basta con pasar la salida completa de la primera capa como entrada a la siguiente.

In [ ]:
class RNNModel(torch.nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.rnn1 = torch.nn.RNN(input_size=28, hidden_size=8, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=8, hidden_size=8, batch_first=True)
        self.rnn3 = torch.nn.RNN(input_size=8, hidden_size=8, batch_first=True)
        self.fc = torch.nn.Linear(in_features=8, out_features=10)
        
    def forward(self, x):
        x = x.squeeze(1)
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out, _ = self.rnn3(out)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

model = RNNModel()
torchsummary.summary(model, input_size=(28, 28))

Hay que tener cuidado porque en Keras el comportamiento es el contrario. Las unidades recurrentes devuelven únicamente el último valor, y es necesario usar el parámetro `return_sequences=True` para que el comportamiento sea el de devolver una secuencia con cada una de las inferencias.

### Entrenamiento del modelo

Por último, entrenaremos nuestra red durante $10$ epochs. Sí, si con las redes recurrentes era lento, con las redes apiladas lo es aún más.

In [ ]:
history = utils.train(
    model=model,
    train_loader=train_loader,
    n_epochs=TRAIN_EPOCHS,
    criterion=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(model.parameters()),
    validation_split=0.1,
    metric_fn=torchmetrics.classification.MulticlassAccuracy(num_classes=10),
)

Veamos cómo ha ido el entrenamiento del modelo.

In [ ]:
pd.DataFrame(history).plot()
plt.yscale('log')
plt.xlabel('Epoch num.')
plt.show()

Quizá la precisión con la que ha clasificado sea mejor, pero en este problema concreto, las redes convolucionales siguen siendo la mejor opción.

### Clasificación de nuevas muestras

Ahora imprimiremos algunos de los elementos de la prueba para ver cómo se han clasificado y cuáles han sido los errores.

In [ ]:
ROWS, COLS = 5, 5
IMAGES = ROWS * COLS

images, labels = [], []
for i in range(IMAGES):
    img, label = test_set[i]
    images.append(img)
    labels.append(label)

images_tensor = torch.stack(images)

model.eval()
with torch.no_grad():
    outputs = model(images_tensor)
    preds = torch.argmax(outputs, dim=1).numpy()

fig = plt.figure(figsize=(15, 15))
for i, (img, true_label, pred_label) in enumerate(zip(images_tensor, labels, preds), 1):
    ax = fig.add_subplot(ROWS, COLS, i)
    img_np = img.squeeze(0).numpy()
    ax.imshow(img_np, cmap='Greens' if true_label == pred_label else 'Reds')
    ax.set_title(f'Expected: {true_label}, Predicted: {pred_label}')
    ax.axis('off')
plt.tight_layout()
plt.show()

## Conclusiones

Hemos mostrado cómo implementar una red neuronal recurrente apilada en PyTorch, concretamente un modelo RNN de dos capas con $8$ unidades en cada capa. Y aunque lo hemos aplicado al problema _fashion MNIST, hemos comprobado que no es la mejor opción (al menos en tiempo de entrenamiento). Sin embargo, apilar RNN puede ser una técnica muy útil para modelar datos secuenciales, ya que permite al modelo capturar dependencias más complejas entre las entradas y las salidas.

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>